# Common import statements
For different splits go to the respective sections.

In [2]:
import pandas as pd
from pandas import DataFrame as df
import IPython as ip
import re

import seaborn as sns
sns.set(color_codes=True)

import matplotlib.pyplot as plt
# Set input dir
import sys
sys.path.append("/home/mschlupp/pythonTools")
files_dir = "files/"

## Split data into `hasEvents` and `noEvents`

In [105]:
ga_full = pd.read_csv(files_dir+"traintest_fullevt.csv")

In [10]:
ga_hasEvts = ga_full[ga_full.hasEvents>0]

In [9]:
ga_noEvts = ga_full[ga_full.hasEvents==0]

In [13]:
ga_noEvts = ga_noEvts.drop(["hasEvents", "nEvts", "longitude_mean"
                , "longitude_variance", "latitude_mean"
                , "latitude_variance", "usageTime_mean"
                , "usageTime_variance", "usageDay_mean"
                ,"usageDay_variance"], axis=1)

In [14]:
ga_noEvts.to_csv("files/ga_noEvts.csv", index=False)

In [15]:
ga_hasEvts.to_csv("files/ga_hasEvts.csv", index=False)

In [17]:
%ls files/

app_events.csv              label_categories.csv
app_labels.csv              phone_brand_device_model.csv
apps_labels_categories.csv  phone_brand_device_model_engl.csv
events.csv                  sample_submission.csv
events_day_hour.csv         traintest_fullevt.csv
ga_hasEvts.csv              traintest_phone.csv
ga_noEvts.csv               traintest_phone_day_hour.csv
gender_age_test.csv         traintest_phone_evts.csv
gender_age_train.csv


## Split events data in ```hasApp``` and ```noApp```

In [130]:
evts= pd.read_csv("files/events_day_hour.csv", index_col=0)

In [3]:
ga_evts = pd.read_csv("files/ga_hasEvts.csv")

There are actually devide_ids that are not in the training and test set, but present in the events data. First get rid of those.

Eg:

In [147]:
-8295895346874348777 in ga_full.device_id.values

False

In [131]:
evts.head(2)

,event_id,device_id,timestamp,longitude,latitude,day,time,hour,usageDay
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,Sun,00:55:25,0,3
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,Sun,00:54:12,0,3


In [151]:
# this is not very fast!
#ll = evts.device_id.map(lambda x: x in ga_evts.device_id.values)
#ll.value_counts()

#True     3160417
#False      92533
#Name: device_id, dtype: int64

In [153]:
# ask if the evts.device_id is in ga_evts
evts=evts.drop(["timestamp"],axis=1)
evts=evts[list(map(lambda x: x in ga_evts.device_id.values,evts.device_id))]

In [155]:
len(evts)

3160417

In [156]:
# Write to csv file
evts.head(2)

,event_id,device_id,longitude,latitude,day,time,hour,usageDay
0,1,29182687948017175,121.38,31.24,Sun,00:55:25,0,3
1,2,-6401643145415154744,103.65,30.97,Sun,00:54:12,0,3


In [157]:
evts.to_csv("files/evts_ga_aligned_ids.csv", index=False)

Ok now all `device_id` entries are present in the train and test samples.

In [158]:
del evts
del evts_iter

In [6]:
#read file in chunks, because else we run in memory troubles
evts_chunks= pd.read_csv("files/evts_ga_aligned_ids.csv", index_col=0, chunksize=5e2)

In [5]:
# this got a bit more complicated than necessary due to missing device ids

# add training flag and write output to file
for i,chunk in enumerate(evts_chunks): 
    l=list()
    for x in chunk.device_id.values:
        l.append(ga_evts[ga_evts.device_id==x].isTrain.values[0])    
    chunk["isTrain"]=l
    #print(type(l), "\n", l)
    
    if i %500 == 0:
        print(i*5e2/1e3, "k events processed!")
    if i==0:
        chunk.to_csv("files/finalSets/evts_withTrain.csv", mode='w')
    else:
        chunk.to_csv("files/finalSets/evts_withTrain.csv", mode='a')

0.0 k events processed!
250.0 k events processed!
